### Get Links from a page on enWikipedia and check if the link is to a Swedish MP person   

  
* This [notebook](https://github.com/salgo60/open-data-examples/blob/master/RiksdagsvoteringGetLinks-enWikipedia.ipynb)
** see also [notebook](https://github.com/salgo60/open-data-examples/blob/master/RiksdagsvoteringGetLinks.ipynb)

version 1 

Lesson learned
* as en:Wikipedia dont have articles of all Swedish MP it gets messy 



In [85]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-12-03 09:55:10.612275


In [86]:
enWikipediaAPIurl = "https://en.wikipedia.org/w/api.php?action=query&format=json&prop=links&meta=&pllimit=500&titles=" 
pagetoCheck = "Andersson_Cabinet"

In [87]:
from IPython.core.display import display, HTML
urlcheck = enWikipediaAPIurl + pagetoCheck
urlArticle = "https://en.wikipedia.org/wiki/" + pagetoCheck


In [88]:
display(HTML("URL sida <a href=" + urlArticle + ">" + pagetoCheck +  "</a><br>* API url <a href=" + urlcheck +">" + urlcheck + "</a>"))

In [93]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
from wikidata.client import Client


In [94]:
def getWD (url):
    print(url)
    baseUrl = "https://en.wikipedia.org/w/api.php?action=query&prop=pageprops|wbentityusage&titles="
    title = url.replace("https://en.wikipedia.org/wiki/","")
    urlWD = baseUrl + title
    wdr = http.request('Get', urlWD + "&format=json",
                 headers={'Content-Type': 'application/json'})

    if wdr.status != 200:
        print("Error ", wdr.status)
        return False
    
    datawd = json.loads(wdr.data.decode('utf-8'),) 
    pg_dict = datawd['query']['pages']
    pg_key = list(pg_dict.keys())[0]
    try:
        wikidata_id = pg_dict[pg_key]['pageprops']['wikibase_item']
    except:
        print("\tError " + url)
        return False
    return wikidata_id

def getRiks(wikidata_id):
    # get SPA 
    client = Client()
    entity = client.get(wikidata_id, load=True)
    try:
        riks_prop = entity[client.get('P8388')]
    except:
        return False
    return riks_prop

In [95]:
urlApi = "https://commons.wikipedia.org/w/api.php"

encoded_body = {
    "action": "query",
    "cmtitle": pagetoCheck,
    "cmlimit": "max",
    "prop": "links",
    "format": "json"
}
print("encoded_body ", type(encoded_body),encoded_body)

r = http.request(method='POST', url=urlApi,
                 headers={'Content-Type': 'application/json'},
                 body=json.dumps(encoded_body))
    
print("Status ",r.status)

#wdr = http.request('Get', urlWD + "&format=json",
#                 headers={'Content-Type': 'application/json'})
    
#datawd = json.loads(wdr.data.decode('utf-8'),) 

#print(r.data)
#values = json.loads(r.data)
#print(values)
#data = json.loads(r.data.decode('utf-8'))    
data = json.loads(r.data.decode('utf-8'))    
links = data["query"]["pages"]
print("pages: ", links)

while('continue' in data):
    print("continue: \t", data['continue'])
    #encoded_body.update(data['plcontinue'])
    r = http.request('POST', urlcheck,
                 headers={'Content-Type': 'application/json'},
                 body=encoded_body)
    data = json.loads(r.data.decode('utf-8'))    
    links = links.update(data['query']['pages'])
    print("Length:",len(links))
print("Endpages: ", len(PAGES), type(PAGES))


encoded_body  <class 'dict'> {'action': 'query', 'cmtitle': 'Andersson_Cabinet', 'cmlimit': 'max', 'prop': 'links', 'format': 'json'}
Status  200


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [96]:
#data

In [28]:
links = data["query"]["pages"]
#print("links: ", type(links),"\t",links.keys(),len(links['7623443']["links"]))
print("Antal länkar: \t",len(links[list(links.keys())[0]]["links"]))

Antal länkar: 	 500


In [30]:
from tqdm.notebook import trange, tqdm
import urllib.parse 

urlen = "en.wikipedia.org/wiki/"
urlwd = "www.wikidata.org/wiki/"
urlriks = "www.riksdagen.se/sv/ledamoter-partier/ledamot/_"
urlriksdata = "data.riksdagen.se/personlista/?utformat=json&iid="

linknr = 0
for link in tqdm(links[list(links.keys())[0]]["links"]):
    #print(link["title"])
    enName = link["title"]
    wdQnumber = getWD (enName)
    wdRiksGUID = getRiks(wdQnumber)
    if wdRiksGUID:
        linknr = linknr + 1
        enUrlArticle = "http://" + urllib.parse.quote(urlen+enName)
        wdQnumberUrl = "http://" + urllib.parse.quote(urlwd+wdQnumber+"#P39")
        riksUrl = "http://" + urllib.parse.quote(urlriks+wdRiksGUID)
        riksData = "http://" + urlriksdata+wdRiksGUID
        display(HTML(str(linknr) + ": <a href="+enUrlArticle+">"+svName+"</a> - <a href="+wdQnumberUrl+">"+wdQnumber+"</a> " \
                + " - <a href="+riksUrl+">Riksdagen</a> " \
                + "<a href="+riksData+">json</a>"))
    else:
        print ("\t missing GUID: \t" + svName) 
               

  0%|          | 0/500 [00:00<?, ?it/s]

14th Government of Slovenia
	 missing GUID: 	A-lotterierna
2018 Swedish general election
	 missing GUID: 	A-lotterierna
2021 Swedish government formation
	 missing GUID: 	A-lotterierna
A-lotterierna
	Error A-lotterierna
	 missing GUID: 	A-lotterierna
Abraham Halef
	Error Abraham Halef
	 missing GUID: 	A-lotterierna
Acko Ankarberg Johansson


Adam Marttinen


Adnan Dibrani


Alexander Christiansson


...

Per Åsling


Pernilla Stålhammar
	Error Pernilla Stålhammar
	 missing GUID: 	A-lotterierna
Persson Cabinet
	 missing GUID: 	A-lotterierna
Peter Helander (politician)
	Error Peter Helander (politician)
	 missing GUID: 	A-lotterierna
Peter Hultqvist


Petter Löberg


Pia Nilsson (politician)


Pia Steensland
	Error Pia Steensland
	 missing GUID: 	A-lotterierna
Poland
	 missing GUID: 	A-lotterierna
Portugal
	 missing GUID: 	A-lotterierna
Posse Cabinet
	Error Posse Cabinet
	 missing GUID: 	A-lotterierna
Prime Minister of Sweden
	 missing GUID: 	A-lotterierna
